In [1]:
#Imports y carga del dataset

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
import nltk
import math
import ast
import io
import re

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

#pd.set_option("display.max_rows", None)

with open("movies_dataset/movies_metadata.csv", "r", encoding='utf-8') as f:
    df_raw = pd.read_csv(f, encoding="utf-8")
    print("Dataset loaded")

[nltk_data] Downloading package stopwords to /home/hector/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/hector/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hector/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Dataset loaded


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
#Vamos a procesar todas las columnas del dataset y construir un dataframe nuevo.
#Las columnas que hemos considerado innecesarias para incluir son:
#"homepage", "imdb_id", "original_title", "poster_path", "status", "video", "id"

exclude = ["homepage", "imdb_id", "original_title", "poster_path", "status", "video", "id", "popularity"] 
df = df_raw.loc[:, df_raw.columns.difference(exclude)]
print("Columnas excluidas")

preprocessed = pd.DataFrame()

Columnas excluidas


In [3]:
#Comprobación de valores nulos

print(pd.isnull(df).any())
print()
print("Filas totales:", len(df))
print("Filas con algún valor nulo:", len([x for x in df.isnull().any(axis=1) if x]))

adult                    False
belongs_to_collection     True
budget                   False
genres                   False
original_language         True
overview                  True
production_companies      True
production_countries      True
release_date              True
revenue                   True
runtime                   True
spoken_languages          True
tagline                   True
title                     True
vote_average              True
vote_count                True
dtype: bool

Filas totales: 45466
Filas con algún valor nulo: 42894


In [4]:
'''ADULT'''

#Aqui observamos que hay valores no booleanos que debemos eliminar, y además observamos que
#las películas con valor True son muy escasas en comparación con las que tienen valor False,
#por lo que vamos a eliminar las filas anómalas y quitar la columna "adult"

print(df["adult"].value_counts())
print()

drop_list = []
for i in range(len(df)):
    if df["adult"][i] != "False":
        drop_list.append(i)

df.drop(index=drop_list, inplace=True)

print(df["adult"].value_counts())

#No añadimos la columna al nuevo dataframe

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: adult, dtype: int64

False    45454
Name: adult, dtype: int64


In [5]:
# Definicion de funciones para el procesamiento de lenguaje natural

replaces = [("i'm", "i am"), ("you're", "you are"), ("we're", "we are"), ("they're", "they are"),
            ("that's", "that is"), ("there's", "there is"), ("who's", "who is"), ("what's", "what is"),
            
            ("isn't", "is not"), ("aren't", "are not"), ("can't", "cannot"), ("haven't", "have not"),
            ("won't", "will not"), ("doesn't", "does not"), ("don't", "do not"), ("didn't", "did not"),
            ("couldn't", "could not"), ("shouldn't", "should not"), ("wasn't", "was not"), ("weren't", "were not"),
            ("hasn't", "has not"), ("hadn't", "had not"),
            
            ("we'd", "we would"), ("he'd", "he would"), ("you'd", "you would"), ("they'd", "they would"), 
            ("who'd", "who would"), ("wouldn't", "would not"),
            
            ("i'll", "i will"), ("you'll", "you will"), ("he'll", "he will"), ("she'll", "she will"),
            ("it'll", "it will"), ("we'll", "we will"), ("they'll", "they will"),
            
            ("i've", "i have"), ("you've", "you have"), ("we've", "we have"), ("they've", "they've"),
            ("&", " and ")
           ]

def sub_contractions(text):
    new_text = text
    for r in replaces:
        new_text = re.sub(f"{r[0]}", f"{r[1]}", new_text)
    return new_text

lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    if type(text) is float:  #Comprobando valores nulos, expresados como un float NaN
        ret_text = []
    else:
        new_text = text.lower()
        new_text = sub_contractions(new_text)
        new_text = re.sub("[^A-Za-zÀ-ÖØ-öø-ÿ0-9 ]", "", new_text)
        
        tok_text = word_tokenize(new_text)
        stop_words = list(stopwords.words("english"))
        tok_text = [word for word in tok_text if word not in stop_words]
        tok_text = [lemmatizer.lemmatize(word) for word in tok_text]
        ret_text = tok_text
        
        """ Processing feedback
        joint_text = " ".join(tok_text)
        print(new_text)
        print(joint_text)
        print() #"""
        
    return ret_text

In [6]:
# title, tagline, overview y belongs_to_collection

In [7]:
'''TITLE'''

preprocessed["title"] = df["title"].apply(preprocess_text)
preprocessed["title"].head()

0                 [toy, story]
1                    [jumanji]
2         [grumpier, old, men]
3            [waiting, exhale]
4    [father, bride, part, ii]
Name: title, dtype: object

In [8]:
'''TAGLINE'''

preprocessed["tagline"] = df["tagline"].apply(preprocess_text)
preprocessed["tagline"].head()

0                                                   []
1                    [roll, dice, unleash, excitement]
2    [still, yelling, still, fighting, still, ready...
3            [friend, people, let, never, let, forget]
4            [world, back, normal, he, surprise, life]
Name: tagline, dtype: object

In [9]:
'''OVERVIEW'''

preprocessed["overview"] = df["overview"].apply(preprocess_text)
preprocessed["overview"].head()

0    [led, woody, andys, toy, live, happily, room, ...
1    [sibling, judy, peter, discover, enchanted, bo...
2    [family, wedding, reignites, ancient, feud, ne...
3    [cheated, mistreated, stepped, woman, holding,...
4    [george, bank, recovered, daughter, wedding, r...
Name: overview, dtype: object

In [10]:
#Fecha dejar a nulo, no eliminar
#Pais / idioma variables categoricas, agrupar en regiones y one hot encoding
#Objetivo es convertir en variables numericas, añadiendo columnas por one hot encoding

In [11]:
'''VOTE_COUNT'''

# Eliminamos las filas con valores nulos
# Si el numero de votos es menor a 1 no es útil, puesto que una pelicula sin votos no sirve
# para deducir en función de la media de votos de una pelicula

preprocessed["vote_count"] = df.drop(df[df.vote_count.isna()].index, inplace=False)["vote_count"].astype("int32")
preprocessed = preprocessed[preprocessed.vote_count >= 1]
preprocessed["vote_count"].head()

0    5415.0
1    2413.0
2      92.0
3      34.0
4     173.0
Name: vote_count, dtype: float64

In [12]:
'''VOTE_AVERAGE'''

# Eliminamos las filas con valores nulos
# De las filas anteriores, es decir, que tengan votos significativos, considerar todos los 
# votos (Hay filas que tienen una media de 0 pero ningun voto, por lo que no son válidas)

preprocessed["vote_average"] = df.drop(df[df.vote_average.isna()].index, inplace=False)["vote_average"]
preprocessed["vote_average"].head()

0    7.7
1    6.9
2    6.5
3    6.1
4    5.7
Name: vote_average, dtype: float64

In [13]:
'''RELEASE_DATE'''

# Eliminamos las filas con valores nulos
# La columna de las fechas estaba en string (probablemente debido a los valores erroneos que había al principio)
# asi que la convertimos en una columna de tipo datetime
# Podemos dejarlos como están o bien agrupar los años para extraer conclusiones

preprocessed["release_date"] = df.drop(df[df.release_date.isna()].index, inplace=False)["release_date"]
preprocessed["release_date"] = pd.to_datetime(preprocessed["release_date"])
preprocessed["release_date"].head()

#df_years = preprocessed['release_date'].groupby(preprocessed.release_date.dt.year)
#print(df_years.first())

0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
Name: release_date, dtype: datetime64[ns]

In [14]:
'''RUNTIME'''

# Eliminamos las filas con valores nulos
# Borrar los que tienen valor 0 porque no tiene sentido que una pelicula tenga 0 minutos 

preprocessed["runtime"] = df.drop(df[df.runtime.isna()].index, inplace=False)["runtime"]
preprocessed = preprocessed[preprocessed.runtime >= 1]
preprocessed["runtime"].head()

0     81.0
1    104.0
2    101.0
3    127.0
4    106.0
Name: runtime, dtype: float64

In [15]:
'''ORIGINAL LANGUAGE'''

#Para elegir los idiomas validos vamos a ver los 6 mas usados y juntar los otros en la categoria resto
#Cambiar filas con idiomas inválidos

idiomas_validos = {'en', 'fr', 'it', 'ja', 'de', 'es'}

def preprocess_language(text):
    if text in idiomas_validos:
        return text
    else:
        return "other"

preprocessed["original_language"] = df["original_language"].apply(preprocess_language)

preprocessed["original_language"].value_counts()

en       29572
other     5195
fr        2226
ja        1282
it        1134
de         927
es         848
Name: original_language, dtype: int64

In [16]:
#Para elegir los idiomas validos vamos a ver los 6 mas usados y hacer un one hot encoding con los que aparezcan


def language_counts(rows):
    languages_dict = {}
    
    for row in rows:
        try:
            languages = ast.literal_eval(row)
            
            for lang in languages:
                code = lang["iso_639_1"]
                
                if code in languages_dict.keys():
                    languages_dict[code] += 1
                else:
                    languages_dict[code] = 1
        except Exception as e:
            continue
            
    sorted_counts = sorted(languages_dict.items(), key=lambda x: x[1], reverse=True)
    return sorted_counts



def treat_languages(df, best_languages):
    for lang in best_languages:
        df[f'spoken_lang_{lang}'] = 0
    df[f'spoken_lang_other'] = 0
    
    for i, row in df.iterrows():
        try:
            language_list = ast.literal_eval(row['spoken_languages'])
            for language_dict in language_list:
                code = language_dict['iso_639_1']
                if code in best_languages:
                    df.iloc[i, df.columns.get_loc(f'spoken_lang_{code}')] = 1
                else:
                    df.iloc[i, df.columns.get_loc(f'spoken_lang_other')] = 1
        except:
            continue


lang_counts = language_counts(df["spoken_languages"])
lang_valid = [lang_counts[i][0] for i in range(7)]
lang_valid.append("other")

preprocessed['spoken_languages'] = df['spoken_languages']
treat_languages(preprocessed, lang_valid)
preprocessed.drop(['spoken_languages'], axis=1, inplace=True)
preprocessed.head()

,title,tagline,overview,vote_count,vote_average,release_date,runtime,original_language,spoken_lang_en,spoken_lang_fr,spoken_lang_de,spoken_lang_es,spoken_lang_it,spoken_lang_ja,spoken_lang_ru,spoken_lang_other
0,"[toy, story]",[],"[led, woody, andys, toy, live, happily, room, ...",5415.0,7.7,1995-10-30,81.0,en,1,0,0,0,0,0,0,0
1,[jumanji],"[roll, dice, unleash, excitement]","[sibling, judy, peter, discover, enchanted, bo...",2413.0,6.9,1995-12-15,104.0,en,1,1,0,0,0,0,0,0
2,"[grumpier, old, men]","[still, yelling, still, fighting, still, ready...","[family, wedding, reignites, ancient, feud, ne...",92.0,6.5,1995-12-22,101.0,en,1,0,0,0,0,0,0,0
3,"[waiting, exhale]","[friend, people, let, never, let, forget]","[cheated, mistreated, stepped, woman, holding,...",34.0,6.1,1995-12-22,127.0,en,1,0,0,0,0,0,0,0
4,"[father, bride, part, ii]","[world, back, normal, he, surprise, life]","[george, bank, recovered, daughter, wedding, r...",173.0,5.7,1995-02-10,106.0,en,1,0,0,0,0,0,0,0


In [17]:
"""
#Para elegir los generos validos vamos a ver los 5 mas usados y juntar los otros en la categoria resto

#print(pd.isnull(df).any())
print(df["genres"].value_counts(dropna=False))

#Camabiar filas con generos inválidos
generos_validos = {'1', '2', '3', '4', '5'}

for nodo in df:
   if genres not in generos_validos 
      genres = other

"""

def genre_counts(rows):
    genre_dict = {}
    
    for row in rows:
        try:
            genres = ast.literal_eval(row)
            
            for genre in genres:
                name = genre["name"]
                
                if name in genre_dict.keys():
                    genre_dict[name] += 1
                else:
                    genre_dict[name] = 1
        except Exception as e:
            continue
            
    sorted_counts = sorted(genre_dict.items(), key=lambda x: x[1], reverse=True)
    return sorted_counts



def treat_genres(df, valid_genres):
    for genre in valid_genres:
        df[genre] = 0
    
    for i, row in df.iterrows():
        try:
            genre_list = ast.literal_eval(row['genres'])
            for genre_dict in genre_list:
                name = genre_dict['name']
                
                if name in valid_genres:
                    df.iloc[i, df.columns.get_loc(genre)] = 1
        except:
            continue



genre_counts = genre_counts(df["genres"])
genre_valid = [genre[0] for genre in genre_counts]

preprocessed['genres'] = df['genres']
treat_genres(preprocessed, genre_valid)
preprocessed.drop(['genres'], axis=1, inplace=True)
preprocessed.head()

,title,tagline,overview,vote_count,vote_average,release_date,runtime,original_language,spoken_lang_en,spoken_lang_fr,...,Family,Mystery,Fantasy,Animation,Foreign,Music,History,War,Western,TV Movie
0,"[toy, story]",[],"[led, woody, andys, toy, live, happily, room, ...",5415.0,7.7,1995-10-30,81.0,en,1,0,...,0,0,0,0,0,0,0,0,0,1
1,[jumanji],"[roll, dice, unleash, excitement]","[sibling, judy, peter, discover, enchanted, bo...",2413.0,6.9,1995-12-15,104.0,en,1,1,...,0,0,0,0,0,0,0,0,0,1
2,"[grumpier, old, men]","[still, yelling, still, fighting, still, ready...","[family, wedding, reignites, ancient, feud, ne...",92.0,6.5,1995-12-22,101.0,en,1,0,...,0,0,0,0,0,0,0,0,0,1
3,"[waiting, exhale]","[friend, people, let, never, let, forget]","[cheated, mistreated, stepped, woman, holding,...",34.0,6.1,1995-12-22,127.0,en,1,0,...,0,0,0,0,0,0,0,0,0,1
4,"[father, bride, part, ii]","[world, back, normal, he, surprise, life]","[george, bank, recovered, daughter, wedding, r...",173.0,5.7,1995-02-10,106.0,en,1,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
#Tenemos que elegir entre tocar o no el presupuesto, si lo dejamos tal cual o en 4 categorias que habia dudas 
#(que sean super produccion, gran produccion, produccion normal y produccion bajo presupuesto por ejemplo)

In [19]:
'''PRODUCTION_COMPANIES'''

# En cuanto a la columna 'production_companies', esta es una columna con diccionarios de compañías,
# por lo que se hará será coger el 'id' de cada compañía productora de la película y contar las apariciones
# de cada compañía para, posteriormente, en esa misma columna poner el valor de las ocurrencias (popularidad)
# máxima de las compañías involucradas.

def popularity_companies(rows):
    companies_popularity = {}
    for row in rows:
        try:
            companies = ast.literal_eval(row)
            for company in companies:
                if company['id'] in companies_popularity.keys():
                    companies_popularity[company['id']] += 1
                else:
                    companies_popularity[company['id']] = 1
        except Exception:
            continue
    return companies_popularity

companies_popularity = popularity_companies(df['production_companies'])

def treat_companies(row):
    max_popularity = 0
    try:
        companies = ast.literal_eval(row)
        for company in companies:
            if int(companies_popularity[company['id']]) > max_popularity:
                max_popularity = companies_popularity[company['id']]
        return max_popularity
    except Exception:
        return max_popularity

In [20]:
preprocessed['production_companies'] = df['production_companies'].apply(treat_companies)
preprocessed['production_companies'].head(20)

0       52
1      197
2     1250
3      836
4      225
5     1250
6     1003
7      263
8      830
9      279
10     431
11     431
12     830
13      84
14    1076
15     830
16     448
17     183
18    1250
19     431
Name: production_companies, dtype: int64

In [21]:
'''PRODUCTION_COUNTRIES'''
# For the 'production_countries', it is a column of dictionary of companies, so the process is going to be take the
# id's of each companies a make an array of ordered id's. The rows that have null values are going to be sustituted
# by empty lists.

def popularity_countries(rows):
    countries_popularity = {}
    
    for row in rows:
        try:
            countries = ast.literal_eval(row)
            for country in countries:
                if country['iso_3166_1'] in countries_popularity.keys():
                    countries_popularity[country['iso_3166_1']] += 1
                else:
                    countries_popularity[country['iso_3166_1']] = 1
        except:
            continue
            
    countries = sorted(countries_popularity.items(), key=lambda x: x[1], reverse=True)[:7]
    best_countries = []
    
    for country in countries:
        best_countries.append(country[0])
        
    return best_countries


def treat_countries(df, best_countries):
    for country in best_countries:
        df[f'production_{country}'] = 0
    df['production_other'] = 0
        
    for i, row in df.iterrows():
        try:
            countries = ast.literal_eval(row['production_countries'])
            for country in countries:
                if country['iso_3166_1'] in best_countries:
                    df.iloc[i, df.columns.get_loc(f'production_{country["iso_3166_1"]}')] = 1
                else:
                    df.iloc[i, df.columns.get_loc('production_other')] = 1
        except:
            continue
            

popular_countries = popularity_countries(df['production_countries'])
preprocessed["production_countries"] = df['production_countries']
treat_countries(preprocessed, popular_countries)
preprocessed.drop(['production_countries'], axis=1, inplace=True)
preprocessed.head()

,title,tagline,overview,vote_count,vote_average,release_date,runtime,original_language,spoken_lang_en,spoken_lang_fr,...,TV Movie,production_companies,production_US,production_GB,production_FR,production_DE,production_IT,production_CA,production_JP,production_other
0,"[toy, story]",[],"[led, woody, andys, toy, live, happily, room, ...",5415.0,7.7,1995-10-30,81.0,en,1,0,...,1,52,1,0,0,0,0,0,0,0
1,[jumanji],"[roll, dice, unleash, excitement]","[sibling, judy, peter, discover, enchanted, bo...",2413.0,6.9,1995-12-15,104.0,en,1,1,...,1,197,1,0,0,0,0,0,0,0
2,"[grumpier, old, men]","[still, yelling, still, fighting, still, ready...","[family, wedding, reignites, ancient, feud, ne...",92.0,6.5,1995-12-22,101.0,en,1,0,...,1,1250,1,0,0,0,0,0,0,0
3,"[waiting, exhale]","[friend, people, let, never, let, forget]","[cheated, mistreated, stepped, woman, holding,...",34.0,6.1,1995-12-22,127.0,en,1,0,...,1,836,1,0,0,0,0,0,0,0
4,"[father, bride, part, ii]","[world, back, normal, he, surprise, life]","[george, bank, recovered, daughter, wedding, r...",173.0,5.7,1995-02-10,106.0,en,1,0,...,1,225,1,0,0,0,0,0,0,0


In [22]:
df['production_countries'][6]

"[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [23]:
'''REVENUE'''
"""
# Lo que haremos será recuperar valores perdidos de esta columna. NO podremos hacerlo mediante los valores anteriores
# ni posteriores, ya que no hay relación temporal en las entradas del dataset. Tampoco utilizaremos la media ya que es
# sensible a los outliers. Por lo que se utilizará la mediana para rellenar.

#Eliminar estos valores o guardarlos en otro dataframe de pruebas
df['revenue'].fillna(df['revenue'].median, inplace=True)
pd.isnull(df).any()"""

preprocessed["revenue"] = df["revenue"]

revenue_valor = preprocessed[preprocessed["revenue"] > 0]
revenue_cero = preprocessed[preprocessed["revenue"] == 0]

In [24]:
print(len(revenue_valor))
print(len(revenue_cero))

7349
33835


In [25]:
revenue_valor.head()

,title,tagline,overview,vote_count,vote_average,release_date,runtime,original_language,spoken_lang_en,spoken_lang_fr,...,production_companies,production_US,production_GB,production_FR,production_DE,production_IT,production_CA,production_JP,production_other,revenue
0,"[toy, story]",[],"[led, woody, andys, toy, live, happily, room, ...",5415.0,7.7,1995-10-30,81.0,en,1,0,...,52,1,0,0,0,0,0,0,0,373554033.0
1,[jumanji],"[roll, dice, unleash, excitement]","[sibling, judy, peter, discover, enchanted, bo...",2413.0,6.9,1995-12-15,104.0,en,1,1,...,197,1,0,0,0,0,0,0,0,262797249.0
3,"[waiting, exhale]","[friend, people, let, never, let, forget]","[cheated, mistreated, stepped, woman, holding,...",34.0,6.1,1995-12-22,127.0,en,1,0,...,836,1,0,0,0,0,0,0,0,81452156.0
4,"[father, bride, part, ii]","[world, back, normal, he, surprise, life]","[george, bank, recovered, daughter, wedding, r...",173.0,5.7,1995-02-10,106.0,en,1,0,...,225,1,0,0,0,0,0,0,0,76578911.0
5,[heat],"[los, angeles, crime, saga]","[obsessive, master, thief, neil, mccauley, lea...",1886.0,7.7,1995-12-15,170.0,en,1,0,...,1250,1,0,0,0,0,0,0,0,187436818.0


In [26]:
revenue_valor.to_csv("movies_training.csv")

In [27]:
revenue_cero.to_csv("movies_testing.csv")

In [28]:
for r in preprocessed:
    print(r)

title
tagline
overview
vote_count
vote_average
release_date
runtime
original_language
spoken_lang_en
spoken_lang_fr
spoken_lang_de
spoken_lang_es
spoken_lang_it
spoken_lang_ja
spoken_lang_ru
spoken_lang_other
Drama
Comedy
Thriller
Romance
Action
Horror
Crime
Documentary
Adventure
Science Fiction
Family
Mystery
Fantasy
Animation
Foreign
Music
History
War
Western
TV Movie
production_companies
production_US
production_GB
production_FR
production_DE
production_IT
production_CA
production_JP
production_other
revenue
